In [125]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import edhec_risk_kit as erk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
hfi = erk.get_hfi_returns().loc["2000":]

In [127]:
hfi.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 227 entries, 2000-01 to 2018-11
Freq: M
Data columns (total 13 columns):
Convertible Arbitrage     227 non-null float64
CTA Global                227 non-null float64
Distressed Securities     227 non-null float64
Emerging Markets          227 non-null float64
Equity Market Neutral     227 non-null float64
Event Driven              227 non-null float64
Fixed Income Arbitrage    227 non-null float64
Global Macro              227 non-null float64
Long/Short Equity         227 non-null float64
Merger Arbitrage          227 non-null float64
Relative Value            227 non-null float64
Short Selling             227 non-null float64
Funds Of Funds            227 non-null float64
dtypes: float64(13)
memory usage: 24.8 KB


In [128]:
hfi.head()

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
2000-01,0.0227,0.0128,0.0088,0.0077,0.0075,0.0088,0.0041,0.0021,0.0075,0.0143,0.0173,0.0427,0.0169
2000-02,0.0267,-0.0022,0.0421,0.0528,0.0253,0.0346,0.0097,0.0408,0.0699,0.0239,0.0185,-0.1340,0.0666
2000-03,0.0243,-0.0138,0.0103,0.0318,0.0134,0.0069,-0.0061,-0.0104,0.0006,0.0131,0.0163,-0.0230,0.0039
2000-04,0.0223,-0.0241,-0.0101,-0.0541,0.0168,-0.0059,-0.0006,-0.0304,-0.0201,0.0188,0.0092,0.1028,-0.0269
2000-05,0.0149,0.0114,-0.0132,-0.0433,0.0062,-0.0034,0.0107,-0.0070,-0.0097,0.0146,0.0080,0.0704,-0.0122


# 1

Use the EDHEC Hedge Fund Indices data set that we used in the lab assignment as well as in the previous week’s assignments. Load them into Python and perform the following analysis based on data since 2000 (including all of 2000): What was the Monthly Parametric Gaussian VaR at the 1% level (as a +ve number) of the Distressed Securities strategy?

In [129]:
erk.var_gaussian(hfi["Distressed Securities"], level=1) * 100

3.1430176939280456

# 2

Use the same data set at the previous question. What was the 1% VaR for the same strategy after applying the Cornish-Fisher Adjustment?

In [130]:
erk.var_gaussian(hfi["Distressed Securities"], level=1, modified=True) * 100

4.966909110627921

# 3

Use the same dataset as the previous question. What was the Monthly Historic VaR at the 1% level (as a +ve number) of the Distressed Securities strategy?

In [131]:
erk.var_historic(hfi["Distressed Securities"], level=1) * 100

4.2642

# 4

Next, load the 30 industry return data using the erk.get_ind_returns() function that we developed during the lab sessions. For purposes of the remaining questions, use data during the 5 year period 2013-2017 (both inclusive) to estimate the expected returns as well as the covariance matrix. To be able to respond to the questions, you will need to build the MSR, EW and GMV portfolios consisting of the “Books”, “Steel”, "Oil", and "Mines" industries. Assume the risk free rate over the 5 year period is 10%.

What is the weight of Steel in the EW Portfolio? 

In [132]:
l = ["books", "steel", "oil", "mines"]

In [133]:
ind = erk.get_ind_returns()
er = erk.annualize_rets(ind["2013":"2017"], 12)[l]
cov = ind["2013":"2017"].cov().loc[l,l]

In [134]:
n = er.shape[0]
w_ew = np.repeat(1/n, n)
r_ew = erk.portfolio_return(w_ew, er)
vol_ew = erk.portfolio_vol(w_ew, cov)

In [135]:
w_ew

array([0.25, 0.25, 0.25, 0.25])

# 5

What is the weight of the largest component of the MSR portfolio? 

In [136]:
erk.msr(.10, er, cov).max()

0.9999999999999953

# 6

Which of the 4 components has the largest weight in the MSR portfolio?

In [137]:
l[erk.msr(.10, er, cov).argmax()]

'steel'

# 7 

How many of the components of the MSR portfolio have non-zero weights?

In [150]:
list(map(my_round, erk.msr(.10, er, cov)))

[0.0, 1.0, 0.0, 0.0]

# 8

What is the weight of the largest component of the GMV portfolio?

In [139]:
erk.gmv(cov).max() * 100

47.702487032166246

# 9

Which of the 4 components has the largest weight in the GMV portfolio? 

In [140]:
l[erk.gmv(cov).argmax()]

'books'

# 10

 How many of the components of the GMV portfolio have non-zero weights? 

In [141]:
erk.gmv(cov)

array([4.77024870e-01, 4.33680869e-19, 4.34112024e-01, 8.88631053e-02])

In [148]:
from functools import partial
my_round = partial(round, ndigits=3)

In [149]:
list(map(my_round, erk.gmv(cov)))

[0.477, 0.0, 0.434, 0.089]

# 11

Assume two different investors invested in the GMV and MSR portfolios at the start of 2018 using the weights we just computed. Compute the annualized volatility of these two portfolios over the next 12 months of 2018? 

(Hint: Use the portfolio_vol code we developed in the lab and use `ind[“2018”][l].cov()` to compute the covariance matrix for 2018, assuming that the variable ind holds the industry returns and the variable l holds the list of industry portfolios you are willing to hold. Don’t forget to annualize the volatility)

In [81]:
er = erk.annualize_rets(ind["2013":"2017"], 12)[l]
cov = ind["2013":"2017"].cov().loc[l,l]

In [82]:
cov

,books,steel,oil,mines
books,0.002720,0.001940,0.001637,0.001539
steel,0.001940,0.004391,0.002122,0.003391
oil,0.001637,0.002122,0.002697,0.002179
mines,0.001539,0.003391,0.002179,0.005232


In [83]:
weights_msr = erk.msr(0.10, er, cov)
weights_msr

array([4.51375048e-15, 1.00000000e+00, 1.40512602e-16, 0.00000000e+00])

In [84]:
weights_gmv = erk.gmv(cov)
weights_gmv

array([4.77024870e-01, 4.33680869e-19, 4.34112024e-01, 8.88631053e-02])

In [110]:
erk.portfolio_vol(weights_msr, ind["2018"].cov().loc[l,l])* 12**0.5 * 100

21.983988181484136

In [111]:
erk.portfolio_vol(weights_gmv, ind["2018"].cov().loc[l,l])* 12**0.5 * 100

18.973734511535802